In [1]:
import numpy as np
import pandas as pd

num = 100

difficulty = np.random.uniform(0, 1, (num,))

speed = np.maximum(np.random.normal(15, 5, (num, )) - difficulty * 10, 0)

accident = np.minimum(np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, (num,)), 0), 1)

df = pd.DataFrame({'difficulty': difficulty, 'speed': speed, 'accident': accident})

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 시뮬레이션 횟수 설정 (많은 반복으로 평균 안정화)
N_sim = 100000  

# 결과를 저장할 리스트
coeffs_q1 = []
coeffs_q2 = []

# 각 시뮬레이션 반복
for _ in range(N_sim):
    # 데이터 생성 (원문 코드 기반)
    num = 100
    difficulty = np.random.uniform(0, 1, (num,))
    speed = np.maximum(np.random.normal(15, 5, (num,)) - difficulty * 10, 0)
    accident = np.minimum(np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, (num,)), 0), 1)
    
    # Question 1: speed를 difficulty로 회귀 (단변량 OLS)
    X1 = sm.add_constant(difficulty)  # 상수항 추가
    model1 = sm.OLS(speed, X1).fit()  # OLS 회귀 fitting
    coeff_q1 = model1.params[1]  # difficulty의 계수
    coeffs_q1.append(coeff_q1)
    
    # Question 2: speed를 difficulty와 accident로 회귀 (다변량 OLS)
    X2 = sm.add_constant(np.column_stack((difficulty, accident)))  # 상수항 + 변수 추가
    model2 = sm.OLS(speed, X2).fit()  # OLS 회귀 fitting
    coeff_q2 = model2.params[1]  # difficulty의 계수 (첫 번째 변수)
    coeffs_q2.append(coeff_q2)

# 평균 계산
avg_q1 = np.mean(coeffs_q1)
avg_q2 = np.mean(coeffs_q2)

# 결과 출력
print(f"Average coefficient for Question 1: {avg_q1}")
print(f"Average coefficient for Question 2: {avg_q2}")

# 옵션 중 가장 가까운 값 찾기 (추가 로직으로 답변 도출)
options_q1 = {'A': -9.661, 'B': -9.243, 'C': -10.14, 'D': -8.927}
options_q2 = {'A': -10.33, 'B': -10.85, 'C': -11.23, 'D': -9.821}

closest_q1 = min(options_q1, key=lambda k: abs(options_q1[k] - avg_q1))
closest_q2 = min(options_q2, key=lambda k: abs(options_q2[k] - avg_q2))

print(f"Closest option for Question 1: {closest_q1}")
print(f"Closest option for Question 2: {closest_q2}")

Average coefficient for Question 1: -9.656416145303814
Average coefficient for Question 2: -10.328108654733013
Closest option for Question 1: A
Closest option for Question 2: A


In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# --- 설정값 ---
# 시뮬레이션 규모를 키워 결과 안정성을 높입니다.
NUM_SAMPLES = 20000
NUM_SIMULATIONS = 2000

def find_closest_option(result, options):
    """
    계산된 결과값(result)과 보기(options)를 비교하여
    가장 가까운 보기의 문자와 값을 반환하는 함수
    """
    best_option_key = None
    min_difference = float('inf')
    
    # 각 옵션과의 차이(절댓값)를 계산
    for key, value in options.items():
        difference = abs(result - value)
        if difference < min_difference:
            min_difference = difference
            best_option_key = key
            
    return best_option_key, options[best_option_key]

def run_simulation(question_number):
    """
    질문 번호에 맞는 시뮬레이션을 실행하고 평균 계수를 반환하는 통합 함수
    """
    coefficients = []
    for _ in range(NUM_SIMULATIONS):
        # 데이터 생성 (공통)
        difficulty = np.random.uniform(0, 1, (NUM_SAMPLES,))
        speed = np.maximum(np.random.normal(15, 5, (NUM_SAMPLES,)) - difficulty * 10, 0)
        
        if question_number == 1:
            # 질문 1: S ~ D
            X = difficulty.reshape(-1, 1)
            y = speed
            model = LinearRegression()
            model.fit(X, y)
            coefficients.append(model.coef_[0])
        elif question_number == 2:
            # 질문 2: S ~ D + A
            accident = np.minimum(np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, (NUM_SAMPLES,)), 0), 1)
            df = pd.DataFrame({'difficulty': difficulty, 'speed': speed, 'accident': accident})
            X = df[['difficulty', 'accident']]
            y = df['speed']
            model = LinearRegression()
            model.fit(X, y)
            coefficients.append(model.coef_[0]) # D의 계수
            
    return np.mean(coefficients)

# --- 메인 코드 실행 ---
if __name__ == "__main__":
    # 각 질문의 보기 정의
    options_q1 = {'A': -9.661, 'B': -9.243, 'C': -10.14, 'D': -8.927}
    options_q2 = {'A': -10.33, 'B': -10.85, 'C': -11.23, 'D': -9.821}

    print("⚙️ 시뮬레이션을 시작합니다. 결과는 실행 시마다 약간의 차이가 있을 수 있습니다.\n")

    # --- 질문 1 실행 및 동적 결과 출력 ---
    result_q1 = run_simulation(question_number=1)
    closest_key_q1, closest_val_q1 = find_closest_option(result_q1, options_q1)
    
    print("--- 질문 1 최종 결과 ---")
    print(f"S ~ D 회귀분석에서 D의 평균 계수: {result_q1:.4f}")
    print(f"▶️ 계산 결과에 따라, 가장 가까운 보기는 {closest_key_q1}) {closest_val_q1} 입니다.\n")

    # --- 질문 2 실행 및 동적 결과 출력 ---
    result_q2 = run_simulation(question_number=2)
    closest_key_q2, closest_val_q2 = find_closest_option(result_q2, options_q2)

    print("--- 질문 2 최종 결과 ---")
    print(f"S ~ D + A 회귀분석에서 D의 평균 계수: {result_q2:.4f}")
    print(f"▶️ 계산 결과에 따라, 가장 가까운 보기는 {closest_key_q2}) {closest_val_q2} 입니다.")

⚙️ 시뮬레이션을 시작합니다. 결과는 실행 시마다 약간의 차이가 있을 수 있습니다.

--- 질문 1 최종 결과 ---
S ~ D 회귀분석에서 D의 평균 계수: -9.6592
▶️ 계산 결과에 따라, 가장 가까운 보기는 A) -9.661 입니다.

--- 질문 2 최종 결과 ---
S ~ D + A 회귀분석에서 D의 평균 계수: -10.3254
▶️ 계산 결과에 따라, 가장 가까운 보기는 A) -10.33 입니다.
